In [1]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from simtk.openmm import app
import mdtraj as md
import os
import pickle
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping


ModuleNotFoundError: No module named 'perses'

# WT RBD:ACE2

In [4]:
# Load PDBs
rbd_pdb = app.PDBFile("../../input/rbd_aligned.pdb")
ace2_pdb = app.PDBFile("../../input/ace2_aligned.pdb")


In [5]:
# Load the ffxmls
ffxml = ['/home/zhangi/choderalab/openmmforcefields/amber/ffxml/protein.ff14SB.xml', "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml", "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/tip3p_standard.xml"]
ff = app.ForceField(*ffxml)

In [6]:
# Add hydrogens to RBD
modeller = app.Modeller(rbd_pdb.topology, rbd_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)

[None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 

In [7]:
ff.createSystem(modeller.topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x2b5dce4df1e0> >

In [8]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("../../input/rbd_protonated.pdb", "w"), keepIds=True)

In [6]:
# Add hydrogens to ACE2
modeller = app.Modeller(ace2_pdb.topology, ace2_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'HID',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 No

In [10]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("../../input/ace2_protonated.pdb", "w"), keepIds=True)

In [21]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_rest_capable_hybrid_topology_factory=False,
                 **kwargs):
        from openeye import oechem

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_file, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

            # Convert positions back to openmm vec3 objects
            complex_positions_vec3 = []
            for position in complex_positions:
                complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
            complex_positions = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))
        
        self.input = inputs
    
    def _solvate(self,
               topology,
               positions,
               water_model,
               phase,
               ionic_strength,
               box_dimensions=None):
        """
        Generate a solvated topology, positions, and system for a given input topology and positions.
        For generating the system, the forcefield files provided in the constructor will be used.
        Parameters
        ----------
        topology : app.Topology
            Topology of the system to solvate
        positions : [n, 3] ndarray of Quantity nm
            the positions of the unsolvated system
        forcefield : SystemGenerator.forcefield
            forcefield file of solvent to add
        water_model : str
            solvent model to use for solvation
        phase : str
            if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
        ionic_strength : float * unit.molar
            the total concentration of ions (both positive and negative) to add using Modeller.
            This does not include ions that are added to neutralize the system.
            Note that only monovalent ions are currently supported.
        Returns
        -------
        solvated_topology : app.Topology
            Topology of the system with added waters
        solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
            Solvated positions
        solvated_system : openmm.System
            The parameterized system, containing a barostat if one was specified.
        """
        modeller = app.Modeller(topology, positions)
    
        geompadding = 0.9 * unit.nanometers
        maxSize = max(max((pos[i] for pos in positions))-min((pos[i] for pos in positions)) for i in range(3))
        vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
        boxVectors = [(maxSize+geompadding)*v for v in vectors]
        
        # Now we have to add missing atoms
        if phase != 'vacuum':
            _logger.info(f"solvating at {ionic_strength} using {water_model}")
            modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
#             if not box_dimensions:
#                 modeller.addSolvent(self.system_generator.forcefield, model=water_model, padding=0.9 * unit.nanometers, ionicStrength=ionic_strength)
#             else:
#                 modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
        else:
            pass

        solvated_topology = modeller.getTopology()
#         if box_dimensions:
#             solvated_topology.setUnitCellDimensions(box_dimensions)
        box_vectors_unitless = [vec.value_in_unit_system(unit.md_unit_system) for vec in boxVectors]
        solvated_topology.setPeriodicBoxVectors(box_vectors_unitless)
        solvated_positions = modeller.getPositions()

        # Canonicalize the solvated positions: turn tuples into np.array
        solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
        solvated_system = self.system_generator.create_system(solvated_topology)

        return solvated_topology, solvated_positions, solvated_system

In [22]:
# Generate htf
solvent_delivery = PointMutationExecutor2("../../input/rbd_protonated.pdb",
                        '1', # First chain is the barstar one
                        '501',
                        'TYR',
                        ligand_input="../../input/ace2_protonated.pdb",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:setup:solvating at 0.15 M using tip3p


In [23]:
solvent_delivery.input[1][0].getPeriodicBoxVectors()

Quantity(value=[Vec3(x=13.556500000000002, y=0.0, z=0.0), Vec3(x=4.518833333333333, y=12.781190772207244, z=0.0), Vec3(x=-4.518833333333333, y=6.390595386103622, z=11.068835899346718)], unit=nanometer)

In [31]:
solvent_delivery.input[1][0].getUnitCellDimensions()

Quantity(value=Vec3(x=13.556500000000002, y=12.781190772207244, z=11.068835899346718), unit=nanometer)

In [27]:
solvent_delivery.input[0][0].getPeriodicBoxVectors()

Quantity(value=[Vec3(x=8.625399999999999, y=0.0, z=0.0), Vec3(x=2.875133333333333, y=8.13210510726193, z=0.0), Vec3(x=-2.875133333333333, y=4.066052553630965, z=7.042609609134006)], unit=nanometer)

In [32]:
solvent_delivery.input[0][0].getUnitCellDimensions()

Quantity(value=Vec3(x=8.625399999999999, y=8.13210510726193, z=7.042609609134006), unit=nanometer)

In [25]:
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in solvent_delivery.input[1][0].getPeriodicBoxVectors()])
box_vectors = np.array([box_vectors])
box_vectors = np.repeat(box_vectors, 100, axis=0)

In [27]:
# box_vectors = solvent_delivery.input[1][0].getPeriodicBoxVectors()
# box_vectors_unitless = [vec.value_in_unit_system(unit.md_unit_system) for vec in box_vectors]

In [30]:
solvent_delivery.input[1][2].getDefaultPeriodicBoxVectors()

[Quantity(value=Vec3(x=13.556500000000002, y=0.0, z=0.0), unit=nanometer),
 Quantity(value=Vec3(x=4.518833333333333, y=12.781190772207244, z=0.0), unit=nanometer),
 Quantity(value=Vec3(x=-4.518833333333333, y=6.390595386103622, z=11.068835899346718), unit=nanometer)]

In [28]:
solvent_delivery.input[0][2].getDefaultPeriodicBoxVectors()

[Quantity(value=Vec3(x=8.625399999999999, y=0.0, z=0.0), unit=nanometer),
 Quantity(value=Vec3(x=2.875133333333333, y=8.13210510726193, z=0.0), unit=nanometer),
 Quantity(value=Vec3(x=-2.875133333333333, y=4.066052553630965, z=7.042609609134006), unit=nanometer)]

In [24]:
app.PDBFile.writeFile(solvent_delivery.input[1][0], solvent_delivery.input[1][1], open("trunc_oct_box_vecs_complex.pdb", "w"), keepIds=True)

In [22]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/113/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

In [2]:
from openmmtools.integrators import LangevinIntegrator
from simtk import openmm
from simtk.openmm import unit, app
import numpy as np

In [ ]:
with open('/data/chodera/zhangi/perses_benchmark/neq/14/113/113_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [ ]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

In [7]:
context = openmm.Context(htf._topology_proposal.new_system, integrator, platform)
context.setPositions(htf.new_positions(htf.hybrid_positions))
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  7.222187772789607e+18 kJ/mol


In [8]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open("/data/chodera/zhangi/perses_benchmark/neq/14/113/complex_new_og.pdb", "w"), keepIds=True)

In [9]:
openmm.LocalEnergyMinimizer.minimize(context)

In [10]:
pos = context.getState(getPositions=True).getPositions()

In [11]:
pos_cleaned = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pos.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)


In [12]:
# new_positions = htf.new_positions(pos_cleaned)

IndexError: index 345591 is out of bounds for axis 0 with size 345591

In [14]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  7.097845652253126e+18 kJ/mol


In [13]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, pos_cleaned, open("/data/chodera/zhangi/perses_benchmark/neq/14/113/complex_new_minimized.pdb", "w"), keepIds=True)

# SA RBD:ACE2

In [2]:
# Load PDBs
rbd_pdb = app.PDBFile("../../input/SA_rbd.pdb")
ace2_pdb = app.PDBFile("../../input/SA_ace2.pdb")


In [3]:
# Load the ffxmls
ffxml = ['/home/zhangi/choderalab/openmmforcefields/amber/ffxml/protein.ff14SB.xml', "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml", "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/tip3p_standard.xml"]
ff = app.ForceField(*ffxml)

In [13]:
# Add hydrogens to RBD
modeller = app.Modeller(rbd_pdb.topology, rbd_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)

[None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 

In [14]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("../../input/SA_rbd_protonated.pdb", "w"), keepIds=True)

In [15]:
# Add hydrogens to ACE2
modeller = app.Modeller(ace2_pdb.topology, ace2_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'HID',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 No

In [16]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("../../input/SA_ace2_protonated.pdb", "w"), keepIds=True)

In [2]:
# Generate htf
solvent_delivery = PointMutationExecutor("../../input/SA_rbd_protonated.pdb",
                        '1', # First chain is the barstar one
                        '501',
                        'ASN',
                        ligand_input="../../input/SA_ace2_protonated.pdb",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD1, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=CE1, atomic number=6), Atom(name=CE2, atomic number=6), Atom(name=CZ, atomic number=6), Atom(name=OH, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HH, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, at

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 5 new atoms
INFO:geometry:	reduced angle potential = 0.9141276538522096.
INFO:geometry:	reduced angle potential = 0.0039249512913720795.
INFO:geometry:	reduced angle potential = 0.3371672712902093.
INFO:geometry:	reduced angle potential = 0.0077215943017904555.
INFO:geometry:	reduced angle potential = 0.1862254214962261.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 1760 bond forces in the no-nonbonded final system
INFO:geometry:	there are 6083 angle forces in the no-nonbonded final system
INFO:geometry:	there are 11515 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system def

...

INFO:geometry:total reduced potential before atom placement: 19165.758376438665


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -7.865087691202411
INFO:geometry:final reduced energy 19160.086700360345
INFO:geometry:sum of energies: 19157.893288747462
INFO:geometry:magnitude of difference in the energies: 2.19341161288225
INFO:geometry:Final logp_proposal: -3506.152949009596


added energy components: [('CustomBondForce', 0.2870428687168419), ('CustomAngleForce', 1.7754427652457068), ('CustomTorsionForce', 2.9720881811427593), ('CustomBondForce', -12.899661506307718)]


INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=11.551200000000001, y=0

In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/115/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

In [2]:
with open('/data/chodera/zhangi/perses_benchmark/neq/14/130/130_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [11]:
modeller = app.Modeller(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions))

In [13]:
modeller.deleteWater()

In [14]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("complex_new_solute.pdb", "w"), keepIds=True)

In [5]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open("complex_new.pdb", "w"), keepIds=True)

In [7]:
for bond in htf._topology_proposal.new_topology.bonds():
    if bond[0].residue.id == '501' or bond[1].residue.id == '501':
        print(bond)

Bond(<Atom 2595 (C) of chain 0 residue 168 (THR)>, <Atom 2605 (N) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2607 (CA) of chain 0 residue 169 (TYR)>, <Atom 2609 (C) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2609 (C) of chain 0 residue 169 (TYR)>, <Atom 2610 (O) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2607 (CA) of chain 0 residue 169 (TYR)>, <Atom 2611 (CB) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2607 (CA) of chain 0 residue 169 (TYR)>, <Atom 2608 (HA) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2605 (N) of chain 0 residue 169 (TYR)>, <Atom 2607 (CA) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2611 (CB) of chain 0 residue 169 (TYR)>, <Atom 2614 (CG) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2611 (CB) of chain 0 residue 169 (TYR)>, <Atom 2612 (HB2) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2611 (CB) of chain 0 residue 169 (TYR)>, <Atom 2613 (HB3) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2605 (N) of chain 0 residue 169 (TYR)>, <Atom 2606 (H) of chain 0 residue 169 (TYR)>)
Bond(<Atom 2609 

In [3]:
from openmmtools.integrators import LangevinIntegrator
from simtk import openmm
from simtk.openmm import unit, app
import numpy as np

In [24]:
with open('/data/chodera/zhangi/perses_benchmark/neq/14/130/130_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [25]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

In [26]:
context = openmm.Context(htf._topology_proposal.new_system, integrator, platform)
context.setPositions(htf.new_positions(htf.hybrid_positions))
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  9.707713448563669e+16 kJ/mol


In [27]:
openmm.LocalEnergyMinimizer.minimize(context)

In [33]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -2965686.201864524 kJ/mol


In [28]:
pos = np.array(context.getState(getPositions=True).getPositions().value_in_unit_system(unit.md_unit_system))

In [29]:
pos

array([[-1.27314508,  1.08152175, -1.03638093],
       [-1.17055011,  1.11668766, -0.97889583],
       [-1.33837831,  0.95153594, -0.99550617],
       ...,
       [-2.10432887,  9.7108984 ,  1.72133681],
       [ 0.54856217,  4.71125793,  1.51521591],
       [-5.87262774,  8.40547848,  5.90232726]])

In [30]:
htf._topology_proposal.new_topology.getNumAtoms()

185792

In [34]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, context.getState(getPositions=True).getPositions(), open("complex_new_minimized.pdb", "w"), keepIds=True)

In [2]:
from openmmtools.integrators import LangevinIntegrator
from simtk import openmm
from simtk.openmm import unit, app
import numpy as np

In [3]:
with open('/data/chodera/zhangi/perses_benchmark/neq/15/145/145_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [4]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

In [5]:
context = openmm.Context(htf._topology_proposal.new_system, integrator, platform)
context.setPositions(htf.new_positions(htf.hybrid_positions))
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  9.707713448563653e+16 kJ/mol


In [6]:
openmm.LocalEnergyMinimizer.minimize(context)

In [7]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -2960235.2913436797 kJ/mol


In [8]:
pos = np.array(context.getState(getPositions=True).getPositions().value_in_unit_system(unit.md_unit_system))

In [9]:
pos

array([[-1.27873445,  1.07064784, -1.12778563],
       [-1.15685022,  1.05656874, -1.12703082],
       [-1.36468315,  0.96350467, -1.06504211],
       ...,
       [-2.08431792,  9.6907835 ,  1.71970596],
       [ 0.52936816,  4.70238018,  1.50693087],
       [-5.89663649,  8.4225359 ,  5.88628811]])

In [10]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, context.getState(getPositions=True).getPositions(), open("/data/chodera/zhangi/perses_benchmark/neq/15/145/complex_new_minimized.pdb", "w"), keepIds=True)

In [11]:
htf._atom_classes['core_atoms']

{2591,
 2592,
 2593,
 2594,
 2595,
 2596,
 2597,
 2598,
 2599,
 2600,
 2601,
 2602,
 2603,
 2604,
 2605,
 2606,
 2607,
 2608,
 2609,
 2610,
 2611,
 2612,
 2613,
 2619,
 2620,
 2621,
 2622,
 2623,
 2624,
 2625}

In [12]:
for atom in htf.hybrid_topology.atoms:
    if atom.residue.resSeq == 501 and atom.residue.name == 'ASN':
        print(atom.index, atom)

2605 ASN501-N
2606 ASN501-H
2607 ASN501-CA
2608 ASN501-HA
2609 ASN501-C
2610 ASN501-O
2611 ASN501-CB
2612 ASN501-HB2
2613 ASN501-HB3
2614 ASN501-CG
2615 ASN501-ND2
2616 ASN501-HD21
2617 ASN501-HD22
2618 ASN501-OD1
185785 ASN501-CD2
185786 ASN501-HD2
185787 ASN501-CG
185788 ASN501-CD1
185789 ASN501-HD1
185790 ASN501-CE1
185791 ASN501-HE1
185792 ASN501-CZ
185793 ASN501-OH
185794 ASN501-HH
185795 ASN501-CE2
185796 ASN501-HE2


In [2]:
with open('/data/chodera/zhangi/perses_benchmark/neq/15/145/145_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [3]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open("/data/chodera/zhangi/perses_benchmark/neq/15/145/complex_new.pdb", "w"), keepIds=True)

In [ ]:
with open('/data/chodera/zhangi/perses_benchmark/neq/15/146/146_complex.pickle', "rb") as f:
    htf = pickle.load(f)

In [ ]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open("/data/chodera/zhangi/perses_benchmark/neq/15/146/complex_new.pdb", "w"), keepIds=True)